In [1]:
import pandas as pd
import nltk; nltk.download('stopwords')
nltk.download('punkt')

from nltk.tokenize import word_tokenize 

# Regular Expressions
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
# import logging
# logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

# import warnings
# warnings.filterwarnings("ignore",category=DeprecationWarning)

# from ngram import NGram

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/tnguy70/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/tnguy70/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
listings = pd.read_csv("listings.csv", usecols=['id', 'name', 'description', 'neighbourhood_cleansed', 'zipcode', 'property_type', 'room_type', 
                                            'price', 'availability_365'])
listings = listings.rename(columns={'id': 'listing_id'})

reviews = pd.read_csv("reviews.csv", usecols=['listing_id', 'comments'])

In [3]:
df = pd.merge(listings, reviews, on='listing_id')

In [4]:
df.describe();

In [5]:
df.head();

In [6]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [7]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(df['name']))

In [8]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['cozy', 'entire', 'floor', 'of', 'brownstone']


In [9]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [10]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)
# print('type(data_words_nostops): ', type(data_words_nostops))

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)
# print('type(data_words_bigrams): ', type(data_words_bigrams))
# print(data_words_bigrams)

# Form Trigrams
data_words_trigrams = make_trigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# ! python3 -m spacy download en

nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_trigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['cozy', 'entire', 'floor', 'brownstone']]


In [11]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1)]]


In [12]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=6, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha=2,
                                           per_word_topics=True)

/software/anaconda3/5.3.0/lib/python3.7/site-packages/gensim/models/ldamodel.py:821: RuntimeWarning: divide by zero encountered in double_scalars
  perwordbound = self.bound(chunk, subsample_ratio=subsample_ratio) / (subsample_ratio * corpus_words)


In [13]:
# Print the Keyword in the 3 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.053*"time" + 0.053*"location" + 0.046*"clean" + 0.042*"luxury" + '
  '0.041*"min" + 0.040*"away" + 0.040*"good" + 0.035*"great" + 0.034*"amazing" '
  '+ 0.033*"huge"'),
 (1,
  '0.289*"apartment" + 0.089*"heart" + 0.048*"village" + 0.046*"renovate" + '
  '0.035*"place" + 0.031*"queen" + 0.029*"newly" + 0.024*"lovely" + '
  '0.023*"space" + 0.022*"oasis"'),
 (2,
  '0.343*"bedroom" + 0.106*"sunny" + 0.084*"home" + 0.065*"large" + '
  '0.049*"side" + 0.032*"park" + 0.025*"low" + 0.020*"brownstone" + '
  '0.019*"friendly" + 0.018*"convenient"'),
 (3,
  '0.142*"spacious" + 0.118*"studio" + 0.095*"cozy" + 0.088*"modern" + '
  '0.087*"beautiful" + 0.049*"new" + 0.044*"quiet" + 0.033*"minute" + '
  '0.030*"subway" + 0.027*"share"'),
 (4,
  '0.421*"room" + 0.389*"private" + 0.049*"bathroom" + 0.047*"comfortable" + '
  '0.018*"single" + 0.015*"entrance" + 0.012*"small" + 0.012*"nice" + '
  '0.004*"rent" + 0.004*"bronx"'),
 (5,
  '0.100*"bed" + 0.048*"bright" + 0.048*"train" + 0.045*"clo

In [14]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -5.590952202839246

Coherence Score:  0.40999205304411146


In [15]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

/software/anaconda3/5.3.0/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.441812 -0.016356       1        1  17.221455
2      0.095854 -0.033038       2        1  16.706219
5      0.089482 -0.148667       3        1  16.637053
3      0.086342 -0.078443       4        1  16.522215
0      0.098449 -0.142142       5        1  16.466757
1      0.071684  0.418646       6        1  16.446299, topic_info=     Category           Freq          Term          Total  loglift  logprob
term                                                                       
10    Default  197728.000000          room  197728.000000  30.0000  30.0000
60    Default  182302.000000       private  182302.000000  29.0000  29.0000
24    Default  156313.000000       bedroom  156313.000000  28.0000  28.0000
7     Default  129427.000000     apartment  129427.000000  27.0000  27.0000
5     Default   64000.000000      spacious   64000.000000  26.0000  26.0000
6     Default   52965.000000        studio   52965.000000  25.0000  25.0000
38    Default   48243.000000         sunny   48243.000000  24.0000  24.0000
40    Default   45352.000000           bed   45352.000000  23.0000  23.0000
1     Default   42584.000000          cozy   42584.000000  22.0000  22.0000
73    Default   39858.000000         heart   39858.000000  21.0000  21.0000
76    Default   39567.000000        modern   39567.000000  20.0000  20.0000
55    Default   39157.000000     beautiful   39157.000000  19.0000  19.0000
51    Default   38280.000000          home   38280.000000  18.0000  18.0000
8     Default   29483.000000         large   29483.000000  17.0000  17.0000
102   Default   23977.000000          time   23977.000000  16.0000  16.0000
66    Default   23754.000000      location   23754.000000  15.0000  15.0000
85    Default   23108.000000      bathroom   23108.000000  14.0000  14.0000
18    Default   22505.000000          side   22505.000000  13.0000  13.0000
168   Default   22116.000000           new   22116.000000  12.0000  12.0000
62    Default   21807.000000        bright   21807.000000  11.0000  11.0000
224   Default   21648.000000         train   21648.000000  10.0000  10.0000
69    Default   21385.000000       village   21385.000000   9.0000   9.0000
70    Default   21819.000000   comfortable   21819.000000   8.0000   8.0000
108   Default   20608.000000         clean   20608.000000   7.0000   7.0000
206   Default   20470.000000      renovate   20470.000000   6.0000   6.0000
178   Default   20507.000000         close   20507.000000   5.0000   5.0000
101   Default   19920.000000         quiet   19920.000000   4.0000   4.0000
106   Default   18915.000000        luxury   18915.000000   3.0000   3.0000
151   Default   18227.000000           min   18227.000000   2.0000   2.0000
200   Default   17872.000000          away   17872.000000   1.0000   1.0000
...       ...            ...           ...            ...      ...      ...
7      Topic6  129426.500000     apartment  129427.328125   1.8051  -1.2418
73     Topic6   39857.882812         heart   39858.710938   1.8050  -2.4196
69     Topic6   21384.589844       village   21385.419922   1.8050  -3.0422
206    Topic6   20469.693359      renovate   20470.521484   1.8050  -3.0859
221    Topic6   15461.655273         place   15462.486328   1.8050  -3.3665
86     Topic6   13725.861328         queen   13726.693359   1.8050  -3.4856
303    Topic6   12776.718750         newly   12777.546875   1.8050  -3.5573
22     Topic6   10815.594727        lovely   10816.425781   1.8050  -3.7239
248    Topic6   10294.852539         space   10295.681641   1.8050  -3.7732
250    Topic6    9657.414062         oasis    9658.245117   1.8050  -3.8372
258    Topic6    9408.769531      charming    9409.601562   1.8050  -3.8632
190    Topic6    8738.428711          cute    8739.260742   1.8050  -3.9371
119    Topic6    8311.568359       stylish    8312.398438   1.8050  -3.9872
243    Topic6    6392.281738 